<a href="https://colab.research.google.com/github/ParhamPishro/Convert-English-Images-to-Farsi/blob/main/Image_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install icrawler
!pip install easyocr
!pip install pyspellchecker
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

In [ ]:
import datetime
import sys
import os,shutil
import re
from icrawler.builtin import google, GoogleImageCrawler
from spellchecker import SpellChecker
from fuzzywuzzy import fuzz
from typing import Literal
from string import ascii_letters
import textwrap
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from deep_translator import GoogleTranslator
import easyocr

In [ ]:
def get_crawled_image_names(parent_path=None):
  list_ = os.listdir(parent_path)
  ext = ['jpg','jpeg','png','bmp']

  filtered = [x for x in list_ if any(y in x for y in ext)]
  
  return filtered

In [ ]:
def generate_html_table(processed_data=None,directory_name=None):

    print("\nGenerateing HTML table started ...")

    if processed_data == None:
         print("Stoped: No processed data found!")
         return
    
    count = len(processed_data)
    str_rows=""
    row_header = "<tr>\n"\
                 "\t<td align='center'><h1>Quote</h1></td>\n"\
                 "\t<td align='center'><h1>Background</h1></td>\n"\
                 "\t<td align='center'><h1>Last File</h1></td>\n"\
                 "</tr>"

    for i in range(count):

          source = processed_data[i]["source_file"]

          wallpaper=processed_data[i]["background"]

          generated=processed_data[i]["generated"]
          row_str ="<tr>\n"\
                   "\t<td><img src='."+source+"' width='300' height='200'/></td>\n"\
                   "\t<td><img src='."+wallpaper+"' width='300' height='200'/></td>\n"\
                   "\t<td><img src='."+generated+"' width='300' height='200'/></td>\n"\
                   "</tr>"
          
          str_rows = str_rows + row_str
    
    print("Collected " +str(count)+" images ...")

    table_str = "<table align='center' border='1'>\n" + row_header + str_rows + "\n</table>"

    html_text = header_text + table_str
    try:
      readme = open(directory_name+"/README.MD", 'w')
      readme.write(html_text)
      readme.close() 

      print("HTML table saved in:", "/README.MD")
    except Exception as err:
      print(f"Unexpected {err=}, {type(err)=}")

    

    print()



In [ ]:
google_crawler = GoogleImageCrawler()
google_crawler.crawl(keyword = "Anne shirley quote", offset = 0, max_num = 20, min_size = None, max_size = None)

In [ ]:
if os.path.exists("/content/anne_quotes"):
  shutil.rmtree("/content/anne_quotes")
os.rename("/content/images", "/content/anne_quotes")

In [ ]:
google_crawler = GoogleImageCrawler()
google_crawler.crawl(keyword = "Background", offset = 0, max_num = 20, min_size = None, max_size = None)

ERROR:downloader:Exception caught when downloading file https://png.png, error: HTTPSConnectionPool(host='png.png', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f675f05e7f0>: Failed to establish a new connection: [Errno -2] Name or service not known')), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://png.png, error: HTTPSConnectionPool(host='png.png', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f675f05e760>: Failed to establish a new connection: [Errno -2] Name or service not known')), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://png.png, error: HTTPSConnectionPool(host='png.png', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f675f05e5e0>: Failed to establish a new conn

In [ ]:
if os.path.exists("/content/images"):
  os.rename("/content/images","/content/backgrounds")

In [ ]:
translator = GoogleTranslator(source='auto', target='fa')
my_font = ImageFont.truetype(font = '/content/BNazanin_0.ttf',size= 50)
processed_images = []

In [ ]:
anne_img_names = get_crawled_image_names("/content/anne_quotes")
background_img_names = get_crawled_image_names("/content/backgrounds")

for i in range(20):
  anne_img_name = "/content/anne_quotes/" + anne_img_names[i]
  background_img_name = "/content/backgrounds/" + background_img_names[i]
  generated_img_name = "/content/output/"+ str(i + 1)+".jpg"

  easyocr_reader = easyocr.Reader(["en"], gpu=False)
  list_of_text = easyocr_reader.readtext(anne_img_name, detail = 0)

  text = str('')
  for item in list_of_text:
    text = text + ' '+ item

  compiled = re.compile(re.escape('anne'), re.IGNORECASE)
  text = compiled.sub('', text)
  compiled = re.compile(re.escape('shirley'), re.IGNORECASE)
  text = compiled.sub('', text)
  text = text.replace('\n',' ')
  text = re.sub('\s+', ' ', text)

  words = text.split()
  final_text=""
  spell = SpellChecker()
  for word in words:
    misspelled = spell.unknown([word])
    if len(misspelled)==1:
      correct = spell.correction(word)
      if correct == None:
        final_text = final_text+ " "+ word
      else:
        final_text = final_text+" "+ spell.correction(word)
    else:
      if final_text =="":
        final_text = word
      else:
        final_text = final_text+ " "+ word

  best_file=""
  best_match = 0
  list_ = os.listdir("/content/books")
  for f in list_:
    path_ = "/content/books" +"/"+ str(f) 
    file = open(path_,"r")
    content = file.read()
    ratio = fuzz.token_set_ratio(final_text, content)
    if best_match < ratio:
      best_match = ratio
      best_file = f

  if best_match < 0.85 :
    continue

  translated = translator.translate(text=final_text)

  out_img = Image.open(background_img_name)
  draw = ImageDraw.Draw(out_img)
  width  = out_img.width
  height = out_img.height
  img_center = (width/2,height/2)
      
  draw.text(xy = img_center,
            text = translated,
            fill="Red",
            stroke_fill="Black",
            stroke_width=10,
            font=my_font,
            direction="rtl",
            language='fa',
            anchor ='mm',
            Literal='center',
            align='right')
  
  out_img.save(generated_img_name)

  anne_img_name = anne_img_name.replace("/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb","")
  background_img_name = background_img_name.replace("/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb","")
  generated_img_name = generated_img_name.replace("/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb","")

  processed_images.append({"source_file":anne_img_name, "background":background_img_name ,"generated":generated_img_name})

  print("Generated image(text match:"+str(best_match)+"%) #",generated_img_name)
  
generate_html_table(processed_data = processed_images, directory_name = "/content/drive/MyDrive/Colab Notebooks/Image_Crawling.ipynb")

Generated image(text match:78%) # /content/output/1.jpg


Generated image(text match:97%) # /content/output/2.jpg


Generated image(text match:82%) # /content/output/3.jpg


Generated image(text match:96%) # /content/output/4.jpg


Generated image(text match:82%) # /content/output/5.jpg


...

Generated image(text match:70%) # /content/output/12.jpg


Generated image(text match:77%) # /content/output/13.jpg


Generated image(text match:79%) # /content/output/14.jpg


Generated image(text match:95%) # /content/output/15.jpg


OSError: ignored

In [ ]:
generate_html_table(processed_data = processed_images, directory_name = "/content/drive/MyDrive/Colab Notebooks/")


Generateing HTML table started ...
Collected 29 images ...
HTML table saved in: /README.MD

